# Helium Revisited:<br> The Variational Principle

The variational principle puts a lower bound on the possible values we can get when solving the Schrodinger equation, since any wavefunction $\psi$ will have to satisfy:

$$ E_{ground} \leq \langle \psi | H| \psi \rangle $$

then we can think about optimizing the $\psi$ to get closer to $E_{ground}$.

Our strategy will be:

* Create a wavefunction $\psi(\zeta)$ as a function of a few parameters (e.g. $\zeta$).
* Create a energy function $E(\zeta)$ that depend on our previous wavefunction.
* Optimize numerically this function over $\zeta$.

<br>
## <i class="fa fa-book"></i>  Preliminaries

In [7]:
from scipy.integrate import quad, dblquad, tplquad
from scipy.optimize import minimize
from scipy.linalg import eigh
import numpy as np

# Part 1: Very Poor Man's Wave-Function

Recall from lecture that Hyllerraas provided a particularly convenient transformation for the inter-electron coordinates in Helium. He wrote that $s = r_1 + r_2$, $ t = r_1 - r_2$, and $u = r_{12}$. This means that our very poor man's (VPM) wave-function goes from
$$ \psi_{VPM}(r_1, r_2) = N e^{-\zeta (r_1 + r_2)} $$
to
$$ \psi_{VPM}(s, t, u) = N e^{-\zeta s}.$$ 

## $\psi(\zeta)$
This is clearly substantially easier to deal with. We've defined this below:

In [8]:
######################## PHI 1 ########################
def phi1(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)

def phi1_du(s, t, u, zeta, Z=2):
    return 0.0

def phi1_dt(s, t, u, zeta, Z=2):
    return 0.0

def phi1_ds(s, t, u, zeta, Z=2):
    return -zeta*np.exp(-zeta*s)


## $E(\zeta)$

Next, we're going to apply the [variational principle](https://en.wikipedia.org/wiki/Variational_method_(quantum_mechanics) like we did in class. We know from lecture that we seek the minimum of $ \langle E(\zeta) \rangle$, where our expectation value is:
$$ \langle E(\zeta) \rangle = \frac{\langle \psi_{VPM}(\zeta) | H | \psi_{VPM}(\zeta) \rangle}{\langle \psi_{VPM}(\zeta) |\psi_{VPM}(\zeta) \rangle} = \frac{H_{11}}{S_{11}}$$


We have worked the math, and have solved for each of these terms, let's program them. Here, you'll want to fill in the two functions below with just the integrands for the  functions above. We'll pass these functions to the triple integration routines next.

$$ H_{11}=\ \langle \psi_{VPM}(\zeta) | H | \psi_{VPM}(\zeta) \rangle = \int_0^\infty ds \int_0^s du \int_0^u dt \; \psi_{VPM}(\zeta)^2 \left( s^2 - t^2 - 4 s u Z + (s - t)(s + t) u \zeta^2 \right )$$


In [15]:
def H11(t, u, s, zeta=1.6875, Z=2):
    return np.exp(-2*s*zeta)*(s*s - t*t - 4.0*s*u*Z + (s - t)*(s + t)*u*zeta*zeta)

$$ S_{11} =\langle \psi_{VPM}(\zeta) |\psi_{VPM}(\zeta) \rangle = \int_0^\infty ds \int_0^s du \int_0^u dt \; u \left( s^2 - t^2 \right) \left(\psi_{VPM}(s; \zeta)\right)^2.$$


In [16]:
def S11(t, u, s, zeta=1.6875, Z=2):
    return u*(s*s - t*t)*phi1(s, t, u, zeta)*phi1(s, t, u, zeta)

This is a 3D integral, so let's go ahead and call the tplquad function (tp is for triple).

I'll give you the code for the overlap integral ($S_{11}$), and you'll have to adopt it to use the function

In [17]:
overlap, error = tplquad( S11  , 
                        0.0, np.inf,
                        lambda x: 0.0, lambda x: x,
                        lambda x, y: 0.0, lambda x, y: y,
                        args=(1.6875, 2.0))
print overlap

0.0216524635072


Adapt the above code to work with the matrix element method ($H_{11}$) you wrote above.

Do you get -0.06165877303407153?

In [18]:
matel, error = tplquad(  H11    , 
                        0.0, 50.0,
                        lambda x: 0.0, lambda x: x,
                        lambda x, y: 0.0, lambda x, y: y,
                        args=(1.6875, 2.0))
print matel

-0.0616587730341


### Define function for $E(\zeta)$
With triple integration in hand, let's write an energy function to feed into our minimizer. You'll want to take the code snippets that you wrote above and paste them into the method, exp_val, below, and then divide them.

In [19]:
def expected_value(zeta):
    # fill me/copy-paste me
    overlap, error = tplquad( S11  , 
                        0.0, np.inf,
                        lambda x: 0.0, lambda x: x,
                        lambda x, y: 0.0, lambda x, y: y,
                        args=(1.6875, 2.0))
    
    matel, error = tplquad( H11     , 
                        0.0, 50.0,
                        lambda x: 0.0, lambda x: x,
                        lambda x, y: 0.0, lambda x, y: y,
                        args=(1.6875, 2.0))
    
    energy = matel/overlap
    return energy

### Optimize $E(\zeta)$
Finally, feed the method that you wrote above and feed it into the **minimize** function that we imported from scipy. 

The syntax is **minimize(function, [starting guess])**.

Use a starting guess of 2.0.

In [21]:
opt = minimize(expected_value,[2.0])
opt

   status: 0
  success: True
     njev: 1
     nfev: 3
 hess_inv: array([[1]])
      fun: -2.8476562500000084
        x: array([ 2.])
  message: 'Optimization terminated successfully.'
      jac: array([ 0.])
      nit: 0

### Compare with the exact energy

In [22]:
Hartree_to_Ev = 27.211399
E_exact=-79
E_var1 = opt['fun']*Hartree_to_Ev
diff_E=E_var1-E_exact
print('Exact Energy --> -79 eV')
print('Variational --> %f eV'%(E_var1))
print('Off by %f eV (%f %%)'%(diff_E, np.abs(diff_E/E_exact)*100))

Exact Energy --> -79 eV
Variational --> -77.488710 eV
Off by 1.511290 eV (1.913025 %)


## Part 2: General Wavefunctions

#### Careful! Many equations are next, you can browse over them, most importantly try to get the gist of them.

Asuming a basic form for wavefunction as:
$$
u^a t^b u^c e^{-\zeta s}
$$
We will assume our wave-function can be given by a sum of the following states:
$$ \phi_1(s,t,u;\zeta) = e^{-\zeta s} $$
$$ \phi_2(s,t,u;\zeta) = e^{-\zeta s}u $$
$$ \phi_3(s,t,u;\zeta) = e^{-\zeta s}t^2 $$
$$ \phi_4(s,t,u;\zeta) = e^{-\zeta s}s $$
$$ \phi_5(s,t,u;\zeta) = e^{-\zeta s}s^2 $$
$$ \phi_6(s,t,u;\zeta) = e^{-\zeta s}u^2 $$
$$ \phi_7(s,t,u;\zeta) = e^{-\zeta s}s u $$
$$ \phi_8(s,t,u;\zeta) = e^{-\zeta s}t^2 u $$
$$ \phi_9(s,t,u;\zeta) = e^{-\zeta s}u^3 $$
$$ \phi_{10}(s,t,u;\zeta) = e^{-\zeta s}t^2 u^2 $$

I've gone ahead and defined these below for you, as well as all their relevant derivatives.
### Just press shift+enter. This is just ugly book keeping.

In [23]:
############################## PHI 2 ########################
def phi2(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)*u

def phi2_du(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)

def phi2_dt(s, t, u, zeta, Z=2):
    return 0.0

def phi2_ds(s, t, u, zeta, Z=2):
    return -zeta*np.exp(-zeta*s)*u

############################## PHI 3 ########################
def phi3(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)*t*t

def phi3_du(s, t, u, zeta, Z=2):
    return 0.0

def phi3_dt(s, t, u, zeta, Z=2):
    return 2.0*np.exp(-zeta*s)*t

def phi3_ds(s, t, u, zeta, Z=2):
    return -zeta*np.exp(-zeta*s)*t*t

############################## PHI 4 ########################
def phi4(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)*s

def phi4_du(s, t, u, zeta, Z=2):
    return 0.0

def phi4_dt(s, t, u, zeta, Z=2):
    return 0.0

def phi4_ds(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)*(1.0 - s*zeta)

############################## PHI 5 ########################
def phi5(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)*s*s

def phi5_du(s, t, u, zeta, Z=2):
    return 0.0

def phi5_dt(s, t, u, zeta, Z=2):
    return 0.0

def phi5_ds(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)*s*(2.0 - s*zeta)

############################## PHI 6 ########################
def phi6(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)*u*u

def phi6_du(s, t, u, zeta, Z=2):
    return 2.0*u*np.exp(-zeta*s)

def phi6_dt(s, t, u, zeta, Z=2):
    return 0.0

def phi6_ds(s, t, u, zeta, Z=2):
    return -zeta*u*u*np.exp(-zeta*s)

############################## PHI 7 ########################
def phi7(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)*s*u

def phi7_du(s, t, u, zeta, Z=2):
    return u*np.exp(-zeta*s)

def phi7_dt(s, t, u, zeta, Z=2):
    return 0.0

def phi7_ds(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)*(u - s*u*zeta)

############################## PHI 8 ########################
def phi8(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)*t*t*u

def phi8_du(s, t, u, zeta, Z=2):
    return t*t*np.exp(-zeta*s)

def phi8_dt(s, t, u, zeta, Z=2):
    return 2.0*t*u*np.exp(-zeta*s)

def phi8_ds(s, t, u, zeta, Z=2):
    return -t*t*u*zeta*np.exp(-zeta*s)

############################## PHI 9 ########################
def phi9(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)*u*u*u

def phi9_du(s, t, u, zeta, Z=2):
    return 3.0*u*u*np.exp(-zeta*s)

def phi9_dt(s, t, u, zeta, Z=2):
    return 0.0

def phi9_ds(s, t, u, zeta, Z=2):
    return -u*u*u*zeta*np.exp(-zeta*s)

############################## PHI 10 ########################
def phi10(s, t, u, zeta, Z=2):
    return np.exp(-zeta*s)*t*t*u*u

def phi10_du(s, t, u, zeta, Z=2):
    return 2.0*t*t*u*np.exp(-zeta*s)

def phi10_dt(s, t, u, zeta, Z=2):
    return 2.0*t*u*u*np.exp(-zeta*s)

def phi10_ds(s, t, u, zeta, Z=2):
    return -t*t*u*u*zeta*np.exp(-zeta*s)

### Important Part: Function Pointers!

In [24]:
phi = {1: (phi1,  phi1_du,  phi1_dt,  phi1_ds ),
       2: (phi2,  phi2_du,  phi2_dt,  phi2_ds ),
       3: (phi3,  phi3_du,  phi3_dt,  phi3_ds ),
       4: (phi4,  phi4_du,  phi4_dt,  phi4_ds ),
       5: (phi5,  phi5_du,  phi5_dt,  phi5_ds ),
       6: (phi6,  phi6_du,  phi6_dt,  phi6_ds ),
       7: (phi7,  phi7_du,  phi7_dt,  phi7_ds ),
       8: (phi8,  phi8_du,  phi8_dt,  phi8_ds ),
       9: (phi9,  phi9_du,  phi9_dt,  phi9_ds ),
       10:(phi10, phi10_du, phi10_dt, phi10_ds)}

Next, we'll need to write down a more general expression for the Hamiltonian matrix elements. Note, that because these functions aren't orthogonal, we're going to have to deal with with off diagonal matrix elements. 

$$ \langle \psi_{n}(\zeta) | H | \psi_{m}(\zeta) \rangle = \int_0^\infty ds \int_0^s du \int_0^u dt \; \left( A_{n,m} + B_{n,m} + C_{n,m} + D_{n,m} \right)$$
where
$$ A_{n,m} = u \left( s^2 - t^2 \right) \left( 
\frac{\partial \phi_n(s,t,u;\zeta)}{\partial s} \frac{\partial \phi_m(s,t,u;\zeta)}{\partial s} + \frac{\partial \phi_n(s,t,u;\zeta)}{\partial t} \frac{\partial \phi_m(s,t,u;\zeta)}{\partial t}+ \frac{\partial \phi_n(s,t,u;\zeta)}{\partial u} \frac{\partial \phi_m(s,t,u;\zeta)}{\partial u} \right)$$

$$ B_{n,m} = s\left(u^2 - t^2 \right) \left( \frac{\partial \phi_n(s, t, u;\zeta) }{\partial u} \frac{\partial \phi_m(s, t, u;\zeta) }{\partial s} + \frac{\phi_n(s, t, u;\zeta) }{\partial s} \frac{\partial \phi_n(s, t, u;\zeta) }{\partial u}\right) $$

$$ C_{n,m} = t\left(s^2 - u^2 \right) \left( \frac{\partial \phi_n(s, t, u;\zeta) }{\partial u} \frac{\partial \phi_m(s, t, u;\zeta) }{\partial t} + \frac{\phi_n(s, t, u;\zeta) }{\partial t} \frac{\partial \phi_n(s, t, u;\zeta) }{\partial u}\right) $$

$$ D_{n,m} = \left( s^2 - t^2 - 4 Z s u \right) \phi_n(s, t, u;\zeta)\phi_m(s, t, u;\zeta).$$

And
$$ \langle \psi_{n}(\zeta) | \psi_{m}(\zeta) \rangle = \int_0^\infty ds \int_0^s du \int_0^u dt \; u \left( s^2 - t^2 \right) \phi_n(s,t,u;\zeta) \phi_m(s,t,u;\zeta).$$

I've gone ahead and defined these for you below because they're a giant pain in the butt to get right.

In [25]:
def Hnm(t, u, s, n, m, zeta=1.6875, Z=2.0):
    A = u*(s*s - t*t)*(phi[n][3](s, t, u, zeta)*phi[m][3](s, t, u, zeta) 
                     + phi[n][2](s, t, u, zeta)*phi[m][2](s, t, u, zeta) 
                     + phi[n][1](s, t, u, zeta)*phi[m][1](s, t, u, zeta))
    
    B = s*(u*u-t*t)*(phi[n][1](s, t, u, zeta)*phi[m][3](s, t, u, zeta)
                   + phi[n][3](s, t, u, zeta)*phi[m][1](s, t, u, zeta))
    
    C = t*(s*s - u*u)*(phi[n][1](s, t, u, zeta)*phi[m][2](s, t, u, zeta)
                     + phi[n][2](s, t, u, zeta)*phi[m][1](s, t, u, zeta))
    
    D = (s*s - t*t - 8.0*s*u)*phi[n][0](s, t, u, zeta)*phi[m][0](s, t, u, zeta)
    return A + B + C + D


def Snm(t, u, s, n, m, zeta=1.6875, Z=2.0):
    prefactor = u*(s*s-t*t)
    return prefactor*phi[n][0](s, t, u, zeta)*phi[m][0](s, t, u, zeta)

## Secular Matrix
Next, let's write a method to create the secular matrix. 

1) The first thing that you're going to do is define the number of functions to include. We're going to start with 2 but this can go up to 10. 

2) Next, you're going to define the Hamiltonian and Overlap matrices to be of size NxN.

3) Write two for-loops to loop through the number of basis functions. For each element of the matrix, use the same code for triple integration that we used above with a few modifications. First, the method that we're integrating is different (Hnm instead of H11, Snm instead of S11), and our args will be args=(n+1, m+1, zeta, 2.0) instead of args=(zeta, 2.0). Why n+1 and m+1?

4) Next, use the hermitian eigenvalue solver to solve the generalized eigenvalue problem. We're going to do this by using eigh(Hmat, Smat).

5) Finally, return the smallest eigenvalue.

With this function written, call minimize on that function with a guess of [1.6875].

In [26]:
def H_sec_gen(zeta):
    nbfs = 2
    Hmat = np.zeros((nbfs, nbfs))
    Smat = np.zeros_like(Hmat)
    
    for n in range(0, nbfs):
        for m in range(0, nbfs):
            Hmat[n, m] = tplquad(Hnm, 
                        0.0, np.inf,
                        lambda x: 0.0, lambda x: x,
                        lambda x, y: 0.0, lambda x, y: y,
                        args=(n+1, m+1, zeta, 2.0))[0]
            Smat[n, m] = tplquad(Snm, 
                        0.0, np.inf,
                        lambda x: 0.0, lambda x: x,
                        lambda x, y: 0.0, lambda x, y: y,
                        args=(n+1, m+1, zeta, 2.0))[0]
    
    evals, evecs = eigh(Hmat, Smat)  
    return np.min(evals)

### Optmize again

This will take some time because we have build a matrix, each matrix element is created via a 3D integration and then we have solve the eigenvalues!

In [27]:
opt = minimize(H_sec_gen,[1.6875])
opt

   status: 0
  success: True
     njev: 4
     nfev: 12
 hess_inv: array([[ 0.80775811]])
      fun: -2.891120716791687
        x: array([ 1.84968448])
  message: 'Optimization terminated successfully.'
      jac: array([ -2.98023224e-08])
      nit: 3

### Compare new results

In [30]:
Hartree_to_Ev = 27.211399
E_exact=-79
E_var1 = opt['fun']*Hartree_to_Ev
diff_E=E_var1-E_exact
print('Exact Energy --> -79 eV')
print('Variational --> %f eV'%(E_var1))
print('Off by %f eV (%f %%)'%(diff_E, np.abs(diff_E/E_exact)*100))

Exact Energy --> -79 eV
Variational --> -78.671439 eV
Off by 0.328561 eV (0.415900 %)


### Questions/Discuss

* How close did you get to the exact value?
* Can you see this working with bigger molecules? 
* What tradeoff/advantages do you see? How could you improve?
* What role do the different functions play in the solution?